# QV质量结果基准

**Quantum Volume**：Quantum Volume是用于衡量量子计算机性能的综合性基准。通过创建特定规格的随机电路来测试，简单的来说，量子计算机设备的量子体积数值是它能够产生可接受结果的最大电路规模的2的幂次方。

**Adjust Heavy Output**：heavy outputs是指概率大于中位概率的输出结果。对于任何电路来说，一半的可能输出都是被定义为heavy outputs。基础目标是希望测量重产出的概率大于三分之二。进一步的，通过调整重输出概率分布公式，引入方差项提高测试的统计显著性。使得测试结果更具有可信度。要求量子体积需要至少运行100个电路，否则测试无效。

基础重输出概率分布: 

$$ \frac{n_h}{n_c n_s}>\frac{2}{3} $$


调整后的重输出概率分布: 

$$ \frac{n_h-z \sqrt{n_h\left(n_s-\frac{n_h}{n_c}\right)}}{n_c n_s}>\frac{2}{3} $$

其中$ n_h $是测量出的重输出次数, $ n_c $是创建的线路数量, $ n_s $是线路测量次数（shots）
Hellinger Distance：概率分布之间的一种度量，通常用于量子态或测量结果之间的比较。在量子计算中，Hellinger Distance 可以用于比较两个量子态之间的相似性。对于两个概率分布 P 和 Q，它们之间的 Hellinger Distance 定义为：

$$ H(P, Q)=\frac{1}{\sqrt{2}} \sqrt{\sum_i\left(\sqrt{p_i}-\sqrt{q_i}\right)^2} $$

数值越低表示两个概率分布越相似，数值越高表示它们之间的差异越大。理想情况下，希望实际测量结果和理论预测之间的 Hellinger Distance 尽可能接近零。

# QV实验默认基础配置

1.电路深度构造方式：方形电路  
2.测量次数shots：5000  
3.批量提交线路数量：100  
4.PulsePeriod： 100  
5.优化、映射：开启  

我们需要确定QV实验的配置参数，才能正确运行QV实验，这些配置参数包括：总量子比特数，量子比特CZ结构（与量子比特总数对应），同时计算批量任务数目，测量次数以及芯片运行参数等。

QV实验的总体代码流程如下：

In [ ]:
import pyqpanda as pq

from originbench.qv_benchmark.qv_bench import QVBench
from originbench.qv_benchmark.qv_options import QVOptions
from originbench.util.options import Options
from originbench.util.chip_topology import CouplingMap

import requests
requests.packages.urllib3.disable_warnings()

#需要先从本源官网获取当前的用户api_token,需要确保有足够的算力资源
my_qcloud_token = "your api token"

def four_qubits_qv_options(cz_pattern, coupling_list):
    
  cz_pattern = cz_pattern
  coupling_list = coupling_list

  coupling_map = CouplingMap(cz_pattern = cz_pattern, couplinglist = coupling_list)

  qv_options = QVOptions()
  qv_options.set_options(circuits_num = 10,
                         qubits_num = 4,
                         shots = 5000,
                       # coupling_map = None,
                         coupling_map = coupling_map,
                         chip_id = 72,
                       # specified_qubits_block = [42, 48],
                         is_amend = True,
                         is_mapping = False,
                         is_optimization = True)

  return qv_options

def run_qv_quality_check(options : QVOptions):
        
  machine = pq.QCloud()
  machine.set_configure(72, 72)
  online_api_key = my_qcloud_token
  machine.init_qvm(online_api_key, False)
  machine.qAlloc_many(72)
  machine.cAlloc_many(72)

  print(options)
  qv_bench = QVBench(options=options)
  
  qv_bench.run(machine)
  qv_bench.analyze()

def run_four_qubits_qv():
    
  cz_pattern_list = [
    [[(12, 13), (7, 8)]],
    [[(25, 31), (37, 36)]],
    [[(30, 31), (31, 37)]],
    [[(42, 48), (49, 50)]],
    [[(49, 55), (61, 67)]],
    [[(61, 67), (68, 69)]],
    [[(61, 62), (68, 69)]]
  ]

  coupling_list_sets = [
    [(12, 13), (13, 7), (7, 8)],
    [(25, 31), (31, 37), (37, 36)],
    [(30, 31), (31, 37), (37, 36)],
    [(42, 48), (48, 49), (49, 50)],
    [(49, 55), (55, 61), (61, 67)],
    [(61, 67), (67, 68), (68, 69)],
    [(61, 62), (62, 68), (68, 69)]
  ]

  # for i in range(len(cz_pattern_list)):
  for i in range(1):
    q4_options = four_qubits_qv_options(cz_pattern=cz_pattern_list[i],
                                        coupling_list=coupling_list_sets[i])
    run_qv_quality_check(q4_options)


if __name__=="__main__":
      
  run_four_qubits_qv()
    

所有计算的结果分析和参数配置信息均会输出到终端控制台上

# 参考文献
[1] https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-hardware/measuring-quantum-volume.ipynb